<a href="https://colab.research.google.com/github/victoriaktruong/scMIXE/blob/main/scGPT_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script preprocesses the primary breast tumor dataset from [Xu et al. (2024)](https://www.cell.com/cell-reports-medicine/fulltext/S2666-3791%2824%2900180-0), which contains 236,363 cells.
The preprocessed data is stored in an AnnData object, which is then used to generate  cell embeddings using [scGPT](https://github.com/bowang-lab/scGPT).

The code was run using **L4 GPU** (Colab Pro).

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing

In [3]:
# Import libraries
!pip install scanpy --quiet

import scanpy as sc
from scipy.io import mmread
import pandas as pd
import numpy as np
import warnings
import json
warnings.filterwarnings("ignore", category=ImportWarning)

In [4]:
# Sparse matrix
matrix = mmread('/content/drive/MyDrive/HuLab/matrix.mtx').tocsr()

# Gene and cell barcodes
genes = pd.read_csv('/content/drive/MyDrive/HuLab/genes.tsv', header=None, sep="\t")
barcodes = pd.read_csv('/content/drive/MyDrive/HuLab/barcodes.tsv', header=None, sep="\t")

print(matrix.shape) # currently prints genes x cells but we want the transpose

(58892, 236363)


In [5]:
matrix = matrix.T

print(matrix.shape)
print(genes.shape)
print(barcodes.shape)

(236363, 58892)
(58892, 2)
(236363, 1)


In [6]:
genes = genes[0]
barcodes = barcodes[0]

print(type(genes))
print(type(barcodes))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [7]:
adata = sc.AnnData(X = matrix)

# Assign gene symbols and barcodes to AnnData
adata.var["gene_symbols"] = genes.values  # Extracted gene names
adata.obs["barcodes"] = barcodes.values   # Extracted barcodes

In [8]:
print(adata.shape)  # Should be (236363, 58892)
print(adata.var.head())
print(adata.obs.head())

(236363, 58892)
   gene_symbols
0  RP11-34P13-3
1       FAM138A
2         OR4F5
3  RP11-34P13-7
4  RP11-34P13-8
                           barcodes
0  Savas_AAACCTGCAAACAACA-1_TIL20_1
1  Savas_AAACCTGCACTACAGT-1_TIL20_1
2  Savas_AAACCTGGTGAGCGAT-1_TIL20_1
3  Savas_AAACCTGTCAAAGACA-1_TIL20_1
4  Savas_AAACGGGAGTGCCAGA-1_TIL20_1


In [9]:
print(adata.obs["barcodes"].nunique() == len(adata.obs["barcodes"]))
print(adata.var["gene_symbols"].nunique() == len(adata.var["gene_symbols"]))

True
True


In [ ]:
# Save the anndata
adata.write('processed_anndata_06-26.h5ad')

# Installing dependencies and setting up the data for training

In [1]:
# Specifically for Google Colab, install dependencies and download data

import os
import sys
%load_ext autoreload
%autoreload 2

if "google.colab" in sys.modules:
    print("Running on Google Colab")
    print("Installing dependencies...")
    !pip install -U scgpt "torch<=2.2.2" "numpy<2" "umap-learn<0.5.7"
    # the optional dependency of flash-attion is skipped on colab
    !pip install wandb louvain

    # NOTE: MAY NEED TO RESTART RUNTIME AFTER THE INSTALLATION

    print("Downloading data and model ckpt...")
    !pip install -q -U gdown
    import gdown

    data_dir = "../../data"
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    if not os.path.exists(os.path.join(data_dir, "Kim2020_Lung.h5ad")):
        !gdown https://drive.google.com/uc?id=1z_0vWYMhRuRiD1EyhuFtY9ReIR0msWaL -O $data_dir/
    if not os.path.exists(os.path.join(data_dir, "covid_subsampled.h5ad")):
        !gdown https://drive.google.com/uc?id=1eD9LbxNJ35YUde3VtdVcjkwm-f4iyJ6x -O $data_dir/
    if not os.path.exists(os.path.join(data_dir, "ms")):
        gdown.download_folder(
            "https://drive.google.com/drive/folders/1Qd42YNabzyr2pWt9xoY4cVMTAxsNBt4v",
            output=os.path.join(data_dir, "ms"),
        )

    print("Downloading model ckpt...")
    model_dir = "../../save/scGPT_human"
    if not os.path.exists(model_dir):
        !mkdir -p $model_dir
        gdown.download_folder(
            #"https://drive.google.com/drive/folders/1oWh_-ZRdhtoGQ2Fw24HP41FgLoomVo-y",
            "https://drive.google.com/drive/folders/1oWh_-ZRdhtoGQ2Fw24HP41FgLoomVo-y?usp=sharing",
            output=model_dir,
        )

Running on Google Colab
Installing dependencies...


In [10]:
from pathlib import Path
import warnings

import scanpy as sc
import scib
import numpy as np
import sys

sys.path.insert(0, "../")

import scgpt as scg
import matplotlib.pyplot as plt
import anndata

plt.style.context('default')
warnings.simplefilter("ignore", ResourceWarning)

#model_dir = Path("../../save/scGPT_human")
model_dir = Path("/content/drive/MyDrive/HuLab/scGPT_human_new")

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.11/dist-packages/scgpt/model/model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/usr/local/lib/python3.11/dist-packages/scgpt/model/multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_

# Prepare the dataset

In [11]:
smaple_data_path = '/content/drive/MyDrive/HuLab/processed_anndata_06-26.h5ad'
adata = sc.read_h5ad(smaple_data_path)

gene_col = "gene_symbols"
cell_type_key = "barcodes"
batch_key = "sample"
N_HVG = 3000

Remove unannotated cells:

In [12]:
# Get the cell type labels as category codes
celltype_id_labels = adata.obs[cell_type_key].astype("category").cat.codes.values

# Count the number of unannotated cells (codes == -1)
num_unannotated = (celltype_id_labels < 0).sum()
print(f"Number of unannotated cells: {num_unannotated}")

# Count the number of annotated cells (codes >= 0)
num_annotated = (celltype_id_labels >= 0).sum()
print(f"Number of annotated cells: {num_annotated}")

# Optional: remove unannotated cells if needed
adata = adata[celltype_id_labels >= 0]


Number of unannotated cells: 0
Number of annotated cells: 236363


Update embedding dimension

In [ ]:
import json

# Load the args.json file from Google Drive
with open('/content/drive/MyDrive/HuLab/scGPT_human_new/args.json', 'r') as f:
    args = json.load(f)

# Change the embedding dimension
args['embsize'] = 768
args['d_hid'] = 768

# Save the updated file BACK to Google Drive
with open('/content/drive/MyDrive/HuLab/scGPT_human_new/args.json', 'w') as f:
    json.dump(args, f, indent=4)

print("Updated embsize to 768 in Google Drive.")


# Generate the scGPT cell embeddings

In [13]:
embed_adata = scg.tasks.embed_data(
    adata,
    model_dir,
    gene_col="gene_symbols",
    batch_size=64,
)

/usr/local/lib/python3.11/dist-packages/scgpt/tasks/cell_emb.py:212: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["id_in_vocab"] = [


scGPT - INFO - match 30865/58892 genes in vocabulary of size 60697.


/usr/local/lib/python3.11/dist-packages/scgpt/model/model.py:77: UserWarning: flash-attn is not installed, using pytorch transformer instead. Set use_fast_transformer=False to avoid this warning. Installing flash-attn is highly recommended.
  warnings.warn(
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
Embedding cells:   0%|          | 0/3694 [00:00<?, ?it/s]<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find

In [14]:
# Inspect the shape of the AnnData object
print(embed_adata.shape)
print(embed_adata.obsm.keys())

# Check the shape of the scGPT embedding matrix
print(embed_adata.obsm['X_scGPT'].shape)

(236363, 30865)
KeysView(AxisArrays with keys: X_scGPT)
(236363, 768)


In [15]:
# Extract scGPT embeddings and save them as a NumPy array
embeddings = embed_adata.obsm["X_scGPT"]
np.save("cell_embeddings_HVG3000_768.npy", embeddings)

# Concatenate scGPT and Mixedbread embeddings

In [ ]:
# Load the embeddings
embeddings = np.load("cell_embeddings_HVG3000_768.npy")
embeddings_mxbai = np.load("embeddings_mxbai.npy")

# Check that both embeddings have the same shape
if embeddings.shape != embeddings_mxbai.shape:
    raise ValueError(f"Shapes do not match: {embeddings.shape} vs {embeddings_mxbai.shape}")

# Perform concatenation
combined_embeddings = np.concatenate((embeddings, embeddings_mxbai), axis=1)

# Check the shape of the result
print(combined_embeddings.shape)  # Expected shape: (236363, 1536)

np.save("concat_emb_mxbai_scgpt_hvg.npy", combined_embeddings)

# Convert `.npy` to `.feather`

In [ ]:
import numpy as np
import pyarrow.feather as feather
import pandas as pd

# Load the cell embeddings
df_scgpt = pd.DataFrame(embeddings)
df_mxbai = pd.DataFrame(embeddings_mxbai)

feather_scgpt = "/content/drive/MyDrive/HuLab/cell_emb_HVG3000_768.feather"
feather_mxbai = "/content/drive/MyDrive/HuLab/embeddings_mxbai.feather"

feather.write_feather(df_scgpt, feather_scgpt)
feather.write_feather(df_mxbai, feather_mxbai)

print(f"Saved embeddings to {feather_scgpt, feather_mxbai}")

In [ ]:
# Load the concatenated cell embeddings
embeddings = np.load('/content/drive/MyDrive/HuLab/Bassez/bassez_scgpt_mxbai_concat_768.npy')
print(embeddings.shape)  # Check the shape of the embeddings



df = pd.DataFrame(embeddings)

feather_path = "bassez_scgpt_mxbai_concat_768.feather"
feather.write_feather(df, feather_path)

print(f"Saved embeddings to {feather_path}")